In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_colwidth', 100)

***Warning this whole script takes 1 hour 45 minutes to run.***

In [2]:
# Use the URL format to get all the pages of the archive

months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 
          'november', 'december']
years = range(2016,2023,1)

nba_urls = []
for y in years:
    for m in months:
        if (m == 'may' and y == 2022):
            break
        nba_urls.append('http://www.espn.com/nba/news/archive/_/month/'+ m +'/year/'+str(y))
        
ncaab_urls = []
for y in years:
    for m in months:
        if (m == 'may' and y == 2022):
            break
        ncaab_urls.append('http://www.espn.com/mens-college-basketball/news/archive?month='+m+'&year='+str(y))

In [3]:
# 15-20 minutes
nba_story_links = []
stories_url_nba = []

# Loop through every link on each archive page
for url in nba_urls:
    # Initialize a counter to count all the articles
    counter = 0
    r = requests.get(url)
    page = BeautifulSoup(r.content, 'html.parser')
    try:
        # If the ul tag exists on a page, scrape all the urls in that list that contain the string 'story'
        for item in page.find('ul').find_all('a'):
            if 'story' in item.get('href'):
                nba_story_links.append(item.get('href'))
                counter += 1
    except AttributeError:
        pass
    stories_url_nba.append((url, counter))
    
# Repeat the same process for men's NCAA articles
ncaab_story_links = []
stories_url_ncaa = []
for url in ncaab_urls:
    counter = 0
    r = requests.get(url)
    page = BeautifulSoup(r.content, 'html.parser')
    try:
        for item in page.find('ul').find_all('a'):
            if 'story' in item.get('href'):
                ncaab_story_links.append(item.get('href'))
                counter += 1
    except AttributeError:
        pass
    stories_url_ncaa.append((url, counter))

In [6]:
# Combine the lists
mbball_stories = nba_story_links
mbball_stories.extend(ncaab_story_links)

In [9]:
# Get the women's basketball archive pages
espnw_links = []
for y in years: 
    for m in months:
        if (m == 'may' and y == 2022):
            break
        espnw_links.append('http://www.espn.com/womens-college-basketball/news/archive?month='+m+'&year='+str(y))

In [10]:
# Repeat the same link getting process
wnba_story_links = []
story_url_wnba = []
for url in espnw_links:
    counter = 0
    r = requests.get(url)
    page = BeautifulSoup(r.content, 'html.parser')
    try: 
        for item in page.find('ul').find_all('a'):
            if 'story' in item.get('href'):
                wnba_story_links.append(item.get('href'))
                counter += 1

    except AttributeError:
        pass
    story_url_wnba.append((url, counter))
        

                

In [12]:
# 45 Minutes

# Initialize lists for article text, authors and dates
w_articles = []
w_authors = []
w_date = []

# Loop through each article link
for url in wnba_story_links:
    r = requests.get(url)
    page = BeautifulSoup(r.content, 'html.parser')
    body = page.find(class_='article-body')
    
    # Append the text to the articles list
    w_articles.append(page.get_text())
    try:
        # If the author class exists, append it
        w_authors.append(page.find(class_='author').get_text())
    except AttributeError:
        w_authors.append(np.nan)
    
    try:
        # If the timestamp class exists, append it
        date = body.find(class_='timestamp')
        w_date.append(date.get_text())
    except AttributeError:
        w_date.append(np.nan)

In [13]:
# Create women's dataframe
dfw = pd.DataFrame({"URL":wnba_story_links, "Author":w_authors, "Date":w_date, "Text":w_articles})

In [14]:
# Make F=1 (changed later when I realized not every article in the women's archive was necessarily about a woman)
dfw['F'] = 1

In [15]:
# Save dataframe to save computation time
dfw.to_csv('espn_wbball_stories.csv')

In [16]:
# Select a random subset of size = the number of women's articles (3577) for the men's articles

# Set random seed
np.random.seed(1680)
m_links = []
for i in range(len(w_articles)):
    # Generate random number to be the index
    U = np.random.randint(0, len(mbball_stories))
    m_links.append(nba_story_links[U])
    # Remove the index so we don't get any duplicates
    mbball_stories.pop(U)
    

In [19]:
# 45 minutes

# Repeat the article scraping process with the random sample of men's articles
m_articles = []
m_authors = []
m_date = []
for url in m_links:
    r = requests.get(url)
    page = BeautifulSoup(r.content, 'html.parser')
    body = page.find(class_='article-body')
    m_articles.append(page.get_text())
    try:
        m_authors.append(page.find(class_='author').get_text())
    except AttributeError:
        m_authors.append(np.nan)
    
    try:
        date = body.find(class_='timestamp')
        m_date.append(date.get_text())
    except AttributeError:
        m_date.append(np.nan)

In [20]:
# Create dataframe of men's articles
dfm = pd.DataFrame({"URL":m_links, "Author":m_authors,"Date":m_date, "Text":m_articles})
dfm['F'] = 0

In [21]:
# Save dataframe to csv
dfm.to_csv('espn_mbball_stories.csv')

In [33]:
print("Total Women's Articles:", len(set(wnba_story_links)))

Total Women's Articles: 3576


In [34]:
print("Total Men's Articles:", len(set(mbball_stories)))

Total Men's Articles: 19809
